In [ ]:
#ライブラリのインポート欄
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
import warnings

from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [ ]:
# CSVを読み込む

train = pd.read_csv('../input/data-science-london-scikit-learn/train.csv', header=None)
test = pd.read_csv('../input/data-science-london-scikit-learn/test.csv', header=None)
target = pd.read_csv('../input/data-science-london-scikit-learn/trainLabels.csv', header=None)
# 提出用に（分ける）
# データの統合（分けて適用するのが面倒なため）

dataset = pd.concat([train, test], ignore_index = True)

In [ ]:
#このプログラムは上書きするので、再実行は上のプログラムからやり直さないとダメ

print('isnull\n',train.isnull().sum())#欠損値がないことを確認

#相関係数のヒートマップを見る

target2 = target.rename(columns={0: 40})
arai = pd.concat([train, target2], axis=1)

sns.heatmap(arai, vmax=1, vmin=-1, center=0)
plt.savefig('seaborn_heatmap_corr_example.png')
print(arai)

In [ ]:
import numpy as np
%matplotlib inline
from sklearn import datasets
from sklearn.model_selection import train_test_split # クロスバリデーション用（テストとトレ分ける）
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC # SVM用
from sklearn import metrics       # 精度検証用
from sklearn.metrics import accuracy_score

X_train, X_test, Y_train, Y_test = train_test_split(train, target, random_state=0) # random_stateはseed値。
print(len(X_train),len(Y_train),len(X_test),len(Y_test))


In [ ]:
# LightGBM
#import lightgbm as lgb
import optuna
import optuna.integration.lightgbm as lgb


# LightGBMのパラメータ設定
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'vervose': 0
}

#binary
#二値分類．ラベルは 0 or 1．
#metricとしては，
#binary_logloss(クロスエントロピー)とbinary_error(正答率)の2つ


# LightGBMを利用するのに必要なフォーマットに変換
lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval = lgb.Dataset(X_test, Y_test, reference=lgb_train)

In [ ]:
# ベストなパラメータ、途中経過を保存する LightGBM学習

best_params, history = {}, []

# LightGBM学習
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=[lgb_train, lgb_eval],
                early_stopping_rounds=50
               )

best_params = gbm.params
best_params


In [ ]:
##ここから推理します

##ここから推理します

import numpy as np
y_pred = gbm.predict(test)# LightGBM推論

Y_pred = np.round(y_pred).astype(np.int32)

# Kaggle提出用csvファイルの作成
#submission = pd.DataFrame({y_pred.astype(np.int32)})
ida = np.arange(1,9001)
submission = pd.DataFrame({"Id": ida, "Solution": Y_pred})
submission.to_csv("submission6.csv", index=False)